#### NCM

In [3]:
import pandas as pd
import numpy as np
import os 
import random
from Funciones import *
from scipy.spatial.distance import cdist
medida = 'euclidean'

random.seed(42)

import pygad 
from sklearn.mixture import GaussianMixture


In [4]:
# Paths and Embeddings for the training set
set = 'Mobio'
a,b = load_train(set)

Img_Con_train_paths = reshape_paths(a[0][0])
Aud_Con_train_paths = reshape_paths(a[0][1])
Mix_Con_train_paths = reshape_paths(a[0][2])

Img_Descon_train_paths = reshape_paths(a[1][0])
Aud_Descon_train_paths = reshape_paths(a[1][1])
Mix_Descon_train_paths = reshape_paths(a[1][2])


Img_Con_train_embeddings = reshape_embed(b[0][0])
Aud_Con_train_embeddings = reshape_embed(b[0][1])
Mix_Con_train_embeddings = reshape_embed(b[0][2])

Img_Descon_train_embeddings = reshape_embed(b[1][0])
Aud_Descon_train_embeddings = reshape_embed(b[1][1])
Mix_Descon_train_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the validation set
a,b = load_validation(set)

Img_Con_validation_paths = reshape_paths(a[0][0])
Aud_Con_validation_paths = reshape_paths(a[0][1])
Mix_Con_validation_paths = reshape_paths(a[0][2])

Img_Descon_validation_paths = reshape_paths(a[1][0])
Aud_Descon_validation_paths = reshape_paths(a[1][1])
Mix_Descon_validation_paths = reshape_paths(a[1][2])


Img_Con_validation_embeddings = reshape_embed(b[0][0])
Aud_Con_validation_embeddings = reshape_embed(b[0][1])
Mix_Con_validation_embeddings = reshape_embed(b[0][2])

Img_Descon_validation_embeddings = reshape_embed(b[1][0])
Aud_Descon_validation_embeddings = reshape_embed(b[1][1])
Mix_Descon_validation_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the test set
a,b = load_test(set)

Img_Con_test_paths = reshape_paths(a[0][0])
Aud_Con_test_paths = reshape_paths(a[0][1])
Mix_Con_test_paths = reshape_paths(a[0][2])

Img_Descon_test_paths = reshape_paths(a[1][0])
Aud_Descon_test_paths = reshape_paths(a[1][1])
Mix_Descon_test_paths = reshape_paths(a[1][2])

Img_Con_test_embeddings = reshape_embed(b[0][0])
Aud_Con_test_embeddings = reshape_embed(b[0][1])
Mix_Con_test_embeddings = reshape_embed(b[0][2])

Img_Descon_test_embeddings = reshape_embed(b[1][0])
Aud_Descon_test_embeddings = reshape_embed(b[1][1])
Mix_Descon_test_embeddings = reshape_embed(b[1][2])



##
Conocidos = len(Img_Con_train_embeddings)
Desconocidos = len(Img_Descon_train_embeddings)
muestras_img = len(Img_Con_train_embeddings[0])
muestras_aud = len(Aud_Con_train_embeddings[0])

img_test = len(Img_Con_test_embeddings[0])
aud_test = len(Aud_Con_test_embeddings[0])

columnas = a[0][0].columns

In [5]:
print ('Known: ', Conocidos)
print ('Unknown: ', Desconocidos)
print ('Images per person (train): ', muestras_img)
print ('Audios per person (train): ', muestras_aud)
print ('Images per person (test): ', img_test)
print ('Audios per person (test): ', aud_test)


Conocidos:  46
Desconocidos:  46
Muestras por imagen:  40
Muestras por audio:  2
Muestras por imagen test:  30
Muestras por audio test:  1


In [6]:
from sklearn.decomposition import PCA

Data_PCA = Mix_Con_train_embeddings
ShapedData = Data_PCA.reshape(Conocidos*muestras_img,1024)
ShapedData = np.array(ShapedData)

PCA_emb = PCA(n_components=.95)
PCA_emb.fit(ShapedData)
dim = PCA_emb.n_components_

### Start of the experiment


In [7]:
def predictor (x,umbral,centroides):    

    distancias = cdist(x, centroides, medida)[0]

    etiquetas = []
    for i in range(len(distancias)):
        if distancias[i]<umbral[i]:
            etiquetas.append(1)
        else:
            etiquetas.append(0)
    
    
    # Since is novelty detection, we only want to know if the person is known or unknown, not who is it
    if sum(etiquetas) == 1:
        
        return True,np.argmax(etiquetas),distancias 
    elif sum (etiquetas) > 1:
        #id = columnas[np.argmin(distancias)]
        id = "muchos"
        return True,id,distancias
    else:
        return False,'Desconocido',distancias

    
    

In [8]:
def ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,muestras_test,umbral,Conocidos_test,centroides):
    
    T_positive = 0 # 
    T_negative = 0 # 
    F_positive = 0 #
    F_negative = 0 #
    
    
    
# We take the samples only for the number of known people we want to use in the test.
    
    con_test = Data_con[:muestras_test*Conocidos_test]
    descon_test = Data_descon
    emb_con_test = emb_con[:muestras_test*Conocidos_test]
    emb_descon_test = emb_descon
    

    # Known test 
    
    for i in range(len(con_test)):
        [resp,id,dist] = predictor(emb_con_test[i],umbral,centroides)
        if resp == True:
            T_positive += 1
            
        if resp == False:
            F_negative += 1
            
      # Uknonwn test

    
    for i in range(len(descon_test)):
        [resp,id,dist] = predictor(emb_descon_test[i],umbral,centroides)
        if resp == True:
            

            F_positive += 1
        if resp == False:
            T_negative += 1
    
    
    
    
    return T_positive,T_negative,F_positive,F_negative

In [9]:
def calc_umbral(Max_distance,Mean_distance,Mat_dist,modo):

    if modo == 0:
        umbrales = [3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))] # Regla de las 3 sigmas, todo mal
    
    if modo == 1:
        umbrales = [np.max(Mean_distance) + 3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))] # El que mejor resultado da diría.
    
    if modo == 2:
        umbrales = [np.mean(Mat_dist[i]) + 3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))] # Efecto que quiere ver javier    
    
    if modo == 3:
        umbrales = [np.max(Mean_distance)+np.std(Mean_distance) for i in range(len(Mat_dist))] # resultados buenos, para imagen
        
    if modo == 4:
        umbrales = [np.max(Mat_dist[i]) for i in range(len(Mat_dist))] # No gusta
        
    if modo == 5:
        umbrales = [np.max(Mean_distance)+3*np.std(Mean_distance) for i in range(len(Mat_dist))]  # Este mas normalizar el total y que de igual coger muchos umbrales es muy bueno en mixto. para PCA es el mejor

    if modo == 6:
        umbrales = [np.mean(Max_distance)+3*np.std(Max_distance) for i in range(len(Mat_dist))] 
    
    if modo == 7:
        umbrales = [np.mean(Max_distance)+3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))]
        
    if modo == 8:
        umbrales = [np.max(Max_distance)+3*np.std(Max_distance) for i in range(len(Mat_dist))]
        
    if modo == 9:
        umbrales = [np.max(Max_distance)-3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))]
        
    if modo == 10:
        umbrales = [np.max(Max_distance)- np.std(Max_distance) for i in range(len(Mat_dist))]
        
    if modo == 11:
        umbrales = [np.max(Mat_dist[i])-3*np.std(Mat_dist[i]) for i in range(len(Mat_dist))]

    return umbrales

In [10]:
def test_incremental_simple (Conocidos_test,calc_umbral,df,Data_con,Data_descon,emb_con,emb_descon,muestras_test,predictor,ensayo,modo = 1,long_emb = 512):
    
      # Thresholds and centroids calculation:


    Data = df.reshape(Conocidos,len(df[0]),long_emb)
        
    Data = np.array(Data)

     # The centroid of each person is calculated as the mean of all their embeddings.

    centroides = np.zeros((Conocidos_test,long_emb))
    for i in range(Conocidos_test):
        centroides[i] = np.mean(Data[i],axis=0)
        
    # We compute the distance of all the embeddings of each person with their centroid. We save it in Dist_mat
    
    # With that we calculate the threshold for each person. We save it in umbrales

    Max_distance = []
    Mean_distance = []
    Dist_mat = []
    for i in range (len(centroides)):
        Dist_mat.append(cdist(centroides[i].reshape(1,-1),Data[i],medida)[0])
        Max_distance.append(np.max(cdist(centroides[i].reshape(1,-1),Data[i],medida)))
        Mean_distance.append(np.mean(cdist(centroides[i].reshape(1,-1),Data[i],medida)))
        
    umbral = calc_umbral(Max_distance,Mean_distance,Dist_mat,modo)

    
    
    T_P, T_N, F_P, F_N = ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,
                                                                           muestras_test,umbral,Conocidos_test,centroides)
    
    return T_P, T_N, F_P, F_N
    
    

In [11]:
def test_incremental_GMM(Conocidos_test,calc_umbral,df,Data_con,Data_descon,emb_con,emb_descon,muestras_test,predictor,ensayo,modo = 1,long_emb = 512,N_gauss = 2):
    
    
    # Insertamos los embeddings en un array
    # Insertamos los embeddings en un array
    Data = df.reshape(Conocidos,len(df[0]),long_emb)
        
  
    Data = np.array(Data)
    ShapedData = Data.reshape(Conocidos_test*muestras_img,1024)
    ShapedData = np.array(ShapedData)
    
    N_gauss = N_gauss*Conocidos_test
    GMM_spherical = GaussianMixture(n_components=N_gauss,covariance_type='spherical',max_iter=1000,random_state=0).fit(ShapedData)
    rangos_GMM = GMM_spherical.precisions_
    centroides_GMM = GMM_spherical.means_
    etiquetas_GMM_sph = GMM_spherical.predict(ShapedData)
    covarianzas_GMM = GMM_spherical.covariances_


    etiquetas_persona_GMM_sph = []
    for i in range(Conocidos_test):
        etiquetas_persona_GMM_sph.append([etiquetas_GMM_sph[i*muestras_img + j] for j in range (muestras_img)])
        
    Max_distance_GMM = []
    Mean_distance_GMM = []
    Dist_mat_GMM = []
    
    etiquetas_persona_GMM_sph = np.array(etiquetas_persona_GMM_sph)
    #return etiquetas_persona_GMM_sph
    

    
    for i in range(len(centroides_GMM)):
        
        Persona = np.unique(np.where(etiquetas_persona_GMM_sph == i)[0])[0] # Persona a la que pertenece el centroide i
        

        Dist_mat_GMM.append(cdist (centroides_GMM[i].reshape(1,-1),Data[Persona],metric = medida))
        Max_distance_GMM.append(np.max(Dist_mat_GMM[i]))
        Mean_distance_GMM.append(np.mean(Dist_mat_GMM[i]))
        
    #     etiq_person = np.unique(etiquetas_persona_GMM_sph[i])
    #     centroides_persona = centroides_GMM[etiq_person]
    #     Dist_mat_GMM.append(cdist (centroides_persona,Data[i],metric = medida))
    #     Max_distance_GMM.append(np.max(Dist_mat_GMM[i]))
    #     Mean_distance_GMM.append(np.mean(Dist_mat_GMM[i]))
     
    # for i in range(len(Dist_mat_GMM)):
    #     Dist_actual = np.array(Dist_mat_GMM[i])
    #     N_centroides = Dist_actual.shape[0]
    #     for j in range(N_centroides):
    #         umbrales = calc_umbral(Max_distance_GMM,Mean_distance_GMM,Dist_mat_GMM,modo)
            
            
    umbral = calc_umbral(Max_distance_GMM,Mean_distance_GMM,Dist_mat_GMM,modo) # Distancias a los umbrales


    T_P, T_N, F_P, F_N = ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,
                                                                            muestras_test,umbral,Conocidos_test,centroides_GMM)
    
    return T_P, T_N, F_P, F_N

In [12]:
result_imagen = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
resultados_imagen = []

for i in range (0,12):
    
    resultados_imagen.append(test_incremental_simple(Conocidos,calc_umbral,Img_Con_train_embeddings,Img_Con_validation_paths,Img_Descon_validation_paths,
                                       Img_Con_validation_embeddings.reshape(Conocidos*img_test,1,512),Img_Descon_validation_embeddings.reshape(Conocidos*img_test,1,512),30,predictor,ensayo,i))
    
    Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*resultados_imagen[i])
    result_imagen.loc[i] = [str(i),Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'Cara']

result_imagen



,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,1.000000,0.047101,0.089965,0.523551,Cara
1,1,0.508856,0.999275,0.674328,0.517391,Cara
2,2,0.673540,0.994203,0.803044,0.756159,Cara
3,3,0.980854,0.965217,0.972973,0.973188,Cara
4,4,0.599038,0.992754,0.747205,0.664130,Cara
5,5,0.816826,0.992029,0.895942,0.884783,Cara
6,6,0.500000,1.000000,0.666667,0.500000,Cara
7,7,0.500181,1.000000,0.666828,0.500362,Cara
8,8,0.500000,1.000000,0.666667,0.500000,Cara
9,9,0.685325,0.984783,0.808207,0.766304,Cara


In [13]:
resultados_audio = []
result_audio = pd.DataFrame(    columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
for i in range (0,12):
    
    resultados_audio.append(test_incremental_simple(Conocidos,calc_umbral,Aud_Con_train_embeddings,Aud_Con_validation_paths,Aud_Descon_validation_paths,
                                       Aud_Con_validation_embeddings.reshape(Conocidos*aud_test,1,512),Aud_Descon_validation_embeddings.reshape(Conocidos*aud_test,1,512),30,predictor,ensayo,i))
    Precision_audio,Recall_audio,F1_audio,Acurracy_audio = calculate_metrics(*resultados_audio[i])
    result_audio.loc[i] = [str(i),Precision_audio,Recall_audio,F1_audio,Acurracy_audio,'Voz']

result_audio

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,0.000000,0.000000,0.000000,0.500000,Voz
1,1,1.000000,0.391304,0.562500,0.695652,Voz
2,2,0.000000,0.000000,0.000000,0.500000,Voz
3,3,0.941176,0.695652,0.800000,0.826087,Voz
4,4,0.000000,0.000000,0.000000,0.500000,Voz
5,5,0.793103,1.000000,0.884615,0.869565,Voz
6,6,1.000000,0.239130,0.385965,0.619565,Voz
7,7,0.000000,0.000000,0.000000,0.500000,Voz
8,8,0.793103,1.000000,0.884615,0.869565,Voz
9,9,1.000000,0.391304,0.562500,0.695652,Voz


In [14]:
resultados_mixto = []
result_mix = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
for i in range (0,12):
    
    resultados_mixto.append(test_incremental_simple(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,i,1024))
    
    Precision_mixto,Recall_mixto,F1_mixto,Acurracy_mixto = calculate_metrics(*resultados_mixto[i])
    result_mix.loc[i] = [str(i),Precision_mixto,Recall_mixto,F1_mixto,Acurracy_mixto,'Cara + Voz']
    
    
result_mix

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,0.000000,0.000000,0.000000,0.500000,Cara + Voz
1,1,1.000000,0.915942,0.956127,0.957971,Cara + Voz
2,2,1.000000,0.474638,0.643735,0.737319,Cara + Voz
3,3,1.000000,0.525362,0.688836,0.762681,Cara + Voz
4,4,1.000000,0.371739,0.541997,0.685870,Cara + Voz
5,5,1.000000,0.892029,0.942934,0.946014,Cara + Voz
6,6,0.972262,0.990580,0.981335,0.981159,Cara + Voz
7,7,1.000000,0.944928,0.971684,0.972464,Cara + Voz
8,8,0.500000,1.000000,0.666667,0.500000,Cara + Voz
9,9,1.000000,0.719565,0.836915,0.859783,Cara + Voz


In [15]:
PCA_Con_train_embeddings = PCA_emb.transform(Mix_Con_train_embeddings.reshape(Conocidos*muestras_img,1024))
PCA_Con_test_embeddings = PCA_emb.transform(Mix_Con_test_embeddings.reshape(Conocidos*img_test,1024))
PCA_Descon_test_embeddings = PCA_emb.transform(Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1024))
PCA_Con_validation_embeddings = PCA_emb.transform(Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1024))
PCA_Descon_validation_embeddings = PCA_emb.transform(Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1024))
resultados_PCA = []
result_PCA = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
for i in range (0,12):
            
    resultados_PCA.append(test_incremental_simple(Conocidos,calc_umbral,PCA_Con_train_embeddings.reshape(Conocidos,muestras_img,dim),Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       PCA_Con_validation_embeddings.reshape(Conocidos*img_test,1,dim),PCA_Descon_validation_embeddings.reshape(Conocidos*img_test,1,dim),
                                       30,predictor,ensayo,i,dim))
    

    Precision_PCA ,Recall_PCA ,F1_PCA ,Acurracy_PCA  = calculate_metrics(*resultados_PCA[i])
    result_PCA.loc[i] = [str(i),Precision_PCA ,Recall_PCA ,F1_PCA ,Acurracy_PCA,'Cara + Voz. PCA']
    
    
result_PCA

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,0.000000,0.000000,0.000000,0.500000,Cara + Voz. PCA
1,1,0.833437,0.975362,0.898831,0.890217,Cara + Voz. PCA
2,2,1.000000,0.917391,0.956916,0.958696,Cara + Voz. PCA
3,3,1.000000,0.943478,0.970917,0.971739,Cara + Voz. PCA
4,4,0.984615,0.881159,0.930019,0.933696,Cara + Voz. PCA
5,5,0.968300,0.973913,0.971098,0.971014,Cara + Voz. PCA
6,6,0.594197,0.994203,0.743833,0.657609,Cara + Voz. PCA
7,7,0.816252,0.975362,0.888742,0.877899,Cara + Voz. PCA
8,8,0.500000,1.000000,0.666667,0.500000,Cara + Voz. PCA
9,9,1.000000,0.952174,0.975501,0.976087,Cara + Voz. PCA


In [16]:
resultados_GMM = []
result_GMM = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
for i in range (0,12):
    print (i)
    
    resultados_GMM.append(test_incremental_GMM(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,i,1024))
    
    Precision_GMM,Recall_GMM,F1_GMM,Acurracy_GMM = calculate_metrics(*resultados_GMM[i])    
    result_GMM.loc[i] = [str(i),Precision_GMM,Recall_GMM,F1_GMM,Acurracy_GMM,'Cara + Voz. GMM']
    
    
    
    
result_GMM


0
1
2
3
4
5
6
7
8
9
10
11


,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,0.838202,0.540580,0.657269,0.718116,Cara + Voz. GMM
1,1,0.500000,1.000000,0.666667,0.500000,Cara + Voz. GMM
2,2,0.500000,1.000000,0.666667,0.500000,Cara + Voz. GMM
3,3,1.000000,0.910870,0.953356,0.955435,Cara + Voz. GMM
4,4,0.929682,0.910145,0.919810,0.920652,Cara + Voz. GMM
5,5,0.963483,0.994203,0.978602,0.978261,Cara + Voz. GMM
6,6,0.506051,1.000000,0.672023,0.511957,Cara + Voz. GMM
7,7,0.500000,1.000000,0.666667,0.500000,Cara + Voz. GMM
8,8,0.500000,1.000000,0.666667,0.500000,Cara + Voz. GMM
9,9,1.000000,0.526087,0.689459,0.763043,Cara + Voz. GMM


In [18]:
resultados = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])
resultados = pd.concat([result_imagen,result_audio,result_mix,result_PCA,result_GMM])
resultados

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,0,1.000000,0.047101,0.089965,0.523551,Cara
1,1,0.508856,0.999275,0.674328,0.517391,Cara
2,2,0.673540,0.994203,0.803044,0.756159,Cara
3,3,0.980854,0.965217,0.972973,0.973188,Cara
4,4,0.599038,0.992754,0.747205,0.664130,Cara
5,5,0.816826,0.992029,0.895942,0.884783,Cara
6,6,0.500000,1.000000,0.666667,0.500000,Cara
7,7,0.500181,1.000000,0.666828,0.500362,Cara
8,8,0.500000,1.000000,0.666667,0.500000,Cara
9,9,0.685325,0.984783,0.808207,0.766304,Cara


In [19]:
resultados.to_csv('Resultados/Resultados_todos_umbrales_clasicos_'+set+'.csv',index=False)

In [20]:
best_umbrales_csv = resultados.groupby('Ensayo').apply(lambda x: x.loc[x['Accuracy'].idxmax()])

In [21]:
best_umbrales_csv

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
Ensayo,,,,,,
Cara,3,0.980854,0.965217,0.972973,0.973188,Cara
Cara + Voz,6,0.972262,0.990580,0.981335,0.981159,Cara + Voz
Cara + Voz. GMM,5,0.963483,0.994203,0.978602,0.978261,Cara + Voz. GMM
Cara + Voz. PCA,9,1.000000,0.952174,0.975501,0.976087,Cara + Voz. PCA
Voz,5,0.793103,1.000000,0.884615,0.869565,Voz


In [22]:
best_umbrales =best_umbrales_csv['Umbral'].values

dict_umbrales ={
    'Cara':int(best_umbrales[0]),
    'Cara + Voz':int(best_umbrales[1]),
    'Cara + Voz. GMM':int(best_umbrales[2]),
    'Cara + Voz. PCA':int(best_umbrales[3]),
    'Voz':int(best_umbrales[4])
}

In [23]:
dict_umbrales
Resultados_Vox = pd.DataFrame(columns=['Umbral','Precision','Recall','F1','Accuracy','Ensayo'])

In [24]:
print ('Cara')
results = test_incremental_simple(Conocidos,calc_umbral,Img_Con_train_embeddings,Img_Con_test_paths,Img_Descon_test_paths,
                                    Img_Con_test_embeddings.reshape(Conocidos*img_test,1,512),Img_Descon_test_embeddings.reshape(Conocidos*img_test,1,512),30,predictor,ensayo,dict_umbrales['Cara'])

Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)

print ('Precision: ',Precision_imagen)
print ('Recall: ',Recall_imagen)
print ('F1: ',F1_imagen)
print ('Acurracy: ',Acurracy_imagen)

Resultados_Vox.loc[0] = [dict_umbrales['Cara'],Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'Cara']

print ('Voz')
results = test_incremental_simple(Conocidos,calc_umbral,Aud_Con_train_embeddings,Aud_Con_test_paths,Aud_Descon_test_paths,
                                    Aud_Con_test_embeddings.reshape(Conocidos*aud_test,1,512),Aud_Descon_test_embeddings.reshape(Conocidos*aud_test,1,512),30,predictor,ensayo,dict_umbrales['Voz'])

Precision_audio,Recall_audio,F1_audio,Acurracy_audio = calculate_metrics(*results)

print ('Precision: ',Precision_audio)
print ('Recall: ',Recall_audio)
print ('F1: ',F1_audio)
print ('Acurracy: ',Acurracy_audio)

Resultados_Vox.loc[1] = [dict_umbrales['Voz'],Precision_audio,Recall_audio,F1_audio,Acurracy_audio,'Voz']

print ('Cara + Voz')

results = test_incremental_simple(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_test_paths,Mix_Descon_test_paths,
                                    Mix_Con_test_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,dict_umbrales['Cara + Voz'],1024)

Precision_mix,Recall_mix,F1_mix,Acurracy_mix = calculate_metrics(*results)

print ('Precision: ',Precision_mix)
print ('Recall: ',Recall_mix)
print ('F1: ',F1_mix)
print ('Acurracy: ',Acurracy_mix)

Resultados_Vox.loc[2] = [dict_umbrales['Cara + Voz'],Precision_mix,Recall_mix,F1_mix,Acurracy_mix,'Cara + Voz']

print ('Cara + Voz. PCA')

results = test_incremental_simple(Conocidos,calc_umbral,PCA_Con_train_embeddings.reshape(Conocidos,muestras_img,dim),Mix_Con_test_paths,Mix_Descon_test_paths,
                                    PCA_Con_test_embeddings.reshape(Conocidos*img_test,1,dim),PCA_Descon_test_embeddings.reshape(Conocidos*img_test,1,dim),30,predictor,ensayo,dict_umbrales['Cara + Voz. PCA'],dim)

Precision_PCA,Recall_PCA,F1_PCA,Acurracy_PCA = calculate_metrics(*results)

print ('Precision: ',Precision_PCA)
print ('Recall: ',Recall_PCA)
print ('F1: ',F1_PCA)
print ('Acurracy: ',Acurracy_PCA)

Resultados_Vox.loc[3] = [dict_umbrales['Cara + Voz. PCA'],Precision_PCA,Recall_PCA,F1_PCA,Acurracy_PCA,'Cara + Voz. PCA']

print ('Cara + Voz. GMM')

results = test_incremental_GMM(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_test_paths,Mix_Descon_test_paths,
                                    Mix_Con_test_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,dict_umbrales['Cara + Voz. GMM'],1024)

Precision_GMM,Recall_GMM,F1_GMM,Acurracy_GMM = calculate_metrics(*results)

print ('Precision: ',Precision_GMM)
print ('Recall: ',Recall_GMM)
print ('F1: ',F1_GMM)
print ('Acurracy: ',Acurracy_GMM)

Resultados_Vox.loc[4] = [dict_umbrales['Cara + Voz. GMM'],Precision_GMM,Recall_GMM,F1_GMM,Acurracy_GMM,'Cara + Voz. GMM']



Cara
Precision:  0.9844097995545658
Recall:  0.9608695652173913
F1:  0.9724972497249725
Acurracy:  0.9728260869565217
Voz
Precision:  0.8518518518518519
Recall:  1.0
F1:  0.92
Acurracy:  0.9130434782608695
Cara + Voz
Precision:  0.9411764705882353
Recall:  0.9971014492753624
F1:  0.9683321604503871
Acurracy:  0.967391304347826
Cara + Voz. PCA
Precision:  0.9970193740685543
Recall:  0.9695652173913043
F1:  0.9831006612784717
Acurracy:  0.9833333333333333
Cara + Voz. GMM
Precision:  0.9785714285714285
Recall:  0.9927536231884058
F1:  0.9856115107913668
Acurracy:  0.9855072463768116


In [25]:
Resultados_Vox.to_csv('Resultados/Resultados_best_umbrales_clasicos_'+set+'.csv',index=False)

In [26]:
Resultados_Vox

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
0,3,0.984410,0.960870,0.972497,0.972826,Cara
1,5,0.851852,1.000000,0.920000,0.913043,Voz
2,6,0.941176,0.997101,0.968332,0.967391,Cara + Voz
3,9,0.997019,0.969565,0.983101,0.983333,Cara + Voz. PCA
4,5,0.978571,0.992754,0.985612,0.985507,Cara + Voz. GMM


In [27]:
best_umbrales_csv

,Umbral,Precision,Recall,F1,Accuracy,Ensayo
Ensayo,,,,,,
Cara,3,0.980854,0.965217,0.972973,0.973188,Cara
Cara + Voz,6,0.972262,0.990580,0.981335,0.981159,Cara + Voz
Cara + Voz. GMM,5,0.963483,0.994203,0.978602,0.978261,Cara + Voz. GMM
Cara + Voz. PCA,9,1.000000,0.952174,0.975501,0.976087,Cara + Voz. PCA
Voz,5,0.793103,1.000000,0.884615,0.869565,Voz
